**Not using Octis as package compatability is poor

In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
import pyLDAvis

import pickle
from pathlib import Path
from datetime import datetime

In [2]:
# load a dataset

dataset_path = Path('../../dataset/topic_modelling/top_10_games/00_Terraria.pkl')

# dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo.pkl')

dataset = pd.read_pickle(dataset_path)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 75499 entries, 63365 to 145140
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         75499 non-null  int64 
 1   app_id        75499 non-null  int64 
 2   app_name      75499 non-null  object
 3   review_text   75499 non-null  object
 4   review_score  75499 non-null  int64 
 5   review_votes  75499 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.0+ MB


In [3]:
# data preprocessing

import re

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)
    
def remove_num(texts):
   output = re.sub(r'\d+', '', texts)
   return output

def unify_whitespaces(x):
    cleaned_string = re.sub(' +', ' ', x)
    return cleaned_string

from nltk.corpus import stopwords
stop=set(stopwords.words("english"))
def remove_stopword(text):
   text=[word.lower() for word in text.split() if word.lower() not in stop]
   return " ".join(text)

# only keep alphabets
def remove_non_alphabets(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    return text

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: clean(x))
    df[review] = df[review].apply(lambda x: deEmojify(x))
    df[review] = df[review].apply(lambda x: remove_num(x))
    df[review] = df[review].apply(lambda x: unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: remove_non_alphabets(x))
    df[review] = df[review].apply(lambda x: remove_stopword(x))
    df[review] = df[review].apply(lambda x: x.lower())

def cleaning_strlist(str_list):
    str_list = list(map(lambda x: clean(x), str_list))
    str_list = list(map(lambda x: deEmojify(x), str_list))
    str_list = list(map(lambda x: remove_num(x), str_list))
    str_list = list(map(lambda x: unify_whitespaces(x), str_list))
    str_list = list(map(lambda x: remove_non_alphabets(x), str_list))
    str_list = list(map(lambda x: remove_stopword(x), str_list))
    str_list = list(map(lambda x: x.lower(), str_list))
    return str_list

In [4]:
# apply data preprocessing

cleaning(dataset, 'review_text')

In [5]:
X = dataset['review_text'].values

In [6]:
X

array(['werewolf riding unicorn shooting rainbows gun build teleporters find hair dresser spider cavern get sword shoots cats take lord moon using yoyo summon sharknado minion shoots sharks enemies find sky temples air wyverns spawn buy music box wizard go record music like playing base whenever want go build castle made entirely white marble would seem thing minecraft game dimension trust get used start learning game terraria simply one satisfying sandbox experiences may sound rude compared minecraft imagination',
       'copies game go around giving people look sad', 'introduction',
       ...,
       'game game start newb get pro hours entertainment even get bored get mods newb',
       'far one greatest games played yet',
       'game awesome eye cthulhu possible'], dtype=object)

In [7]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [8]:
# lemmatize the data

X_lemmatized = list(map(lambda x: lemmatization(x), X))

In [10]:
# save the lematized data, as separate pickle file

import pickle

X_lemmatized_file = Path('00_Terraria_lemmatized.pkl')

with open(X_lemmatized_file, "wb") as f:
    pickle.dump(X_lemmatized, f)

In [11]:
X_lemmatized_file = Path('00_Terraria_lemmatized.pkl')

with open(X_lemmatized_file, "rb") as f:
    X_lemmatized = pickle.load(f)

X_lemmatized[0]

['werewolf',
 'rid',
 'unicorn',
 'shooting',
 'rainbow',
 'gun',
 'build',
 'teleporters',
 'find',
 'hair',
 'dresser',
 'spider',
 'cavern',
 'get',
 'sword',
 'shoot',
 'cat',
 'take',
 'lord',
 'moon',
 'use',
 'yoyo',
 'summon',
 'sharknado',
 'minion',
 'shoot',
 'sharks',
 'enemy',
 'find',
 'sky',
 'temple',
 'air',
 'wyverns',
 'spawn',
 'buy',
 'music',
 'box',
 'wizard',
 'go',
 'record',
 'music',
 'like',
 'play',
 'base',
 'whenever',
 'want',
 'go',
 'build',
 'castle',
 'make',
 'entirely',
 'white',
 'marble',
 'would',
 'seem',
 'thing',
 'minecraft',
 'game',
 'dimension',
 'trust',
 'get',
 'use',
 'start',
 'learning',
 'game',
 'terrarium',
 'simply',
 'one',
 'satisfy',
 'sandbox',
 'experience',
 'may',
 'sound',
 'rude',
 'compare',
 'minecraft',
 'imagination']

In [12]:
# use gensim to build a dictionary and train our LDAModel

id2word = gensim.corpora.Dictionary(X_lemmatized)

corpus = [id2word.doc2bow(text) for text in X_lemmatized]

In [15]:
%load_ext autoreload

In [16]:
%autoreload 2

from lda_multicore_octis import LDAMulticoreOctis
from octis_dataset_handler import OctisDatasetHandler

In [17]:
# define the dataset in Octis
dataset_handler = OctisDatasetHandler(corpus=corpus)
# dataset_handler.set_corpus(corpus)

In [18]:
dataset_handler.get_corpus()[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 2),
 (15, 2),
 (16, 2),
 (17, 2),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 2),
 (28, 1),
 (29, 1),
 (30, 2),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 2),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 2),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1)]

In [25]:
# load the optimizer for hyperparameter optimization

%autoreload 2
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence

search_space = {"num_topics":  Categorical({10, 20, 30}),
                "decay": Categorical({0.5, 0.6, 0.7}),
                "offset": Categorical({1, 4, 16, 64, 256, 1024})}

In [21]:
npmi = Coherence(texts=X_lemmatized)

In [22]:
optimization_runs=30
model_runs=1

In [23]:
N_TOPICS = 20

lda_model = LDAMulticoreOctis(corpus=corpus,
                              id2word=id2word,
                              num_topics=N_TOPICS,
                              random_state=42,
                              chunksize=2048,
                              workers=3, passes=10)

# lda_model.train_model(dataset_handler)

In [26]:
optimizer=Optimizer()
optimization_result = optimizer.optimize(
    lda_model, dataset_handler, npmi, search_space, number_of_call=optimization_runs, 
    model_runs=model_runs, save_models=True, 
    extra_metrics=None, # to keep track of other metrics
    save_path='results/test_lda/')

ValueError: Provided transformers should be a Transformer instance. Got <skopt.space.transformers.LabelEncoder object at 0x7f4503508790>